In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
% matplotlib inline

In [2]:
import numpy as np
import sys
sys.path.append('..')
from mlscratch.nn_layers import fully_connected


class NeuralNetwork(object):

    def __init__(self):
        self.architecture = []
        self.weights = [] # TODO: change to np.array() blank array if possible. # sudo code
        self.loss_function = None
        self.optimizer = None
        
    def add(self, layer):
        self.architecture.append(layer)

    def compile(self, input_size):
        self.architecture.insert(0, fully_connected(num_nodes=input_size, activation=None))
        for i in range(0, len(self.architecture)-1):
            self.weights.append(self._weight_initialization(self.architecture[i], self.architecture[i+1]))
    def train(self, x, y, batch_size):
        pass

    def validate(self, x, y):
        pass

    def predict(self, x):
        pass

    def _forward_pass(self):
        pass

    def _backward_pass(self):
        pass
    
    def _weight_initialization(self, layer_l_minus1, layer_l):
        num_nodes_layer_l_minus1 = layer_l_minus1.num_nodes
        num_nodes_layer_l = layer_l.num_nodes
        weight_matrix = np.random.randn(num_nodes_layer_l, num_nodes_layer_l_minus1) * \
                  np.sqrt(2/num_nodes_layer_l_minus1)
        return weight_matrix

In [3]:
# sudo code
from mlscratch.nn_layers import fully_connected
from mlscratch.optimization import adam
from mlscratch.activation import relu, sigmoid
from mlscratch.loss_functions import binary_cross_entropy

neural_network = NeuralNetwork()
neural_network.add(layer=fully_connected(num_nodes=10, activation=relu))
neural_network.add(layer=fully_connected(num_nodes=10, activation=relu))
neural_network.add(layer=fully_connected(num_nodes=1, activation=sigmoid))
neural_network.loss_function = binary_cross_entropy
neural_network.optimizer = adam

In [4]:
x = np.array(np.arange(100))
delta = np.random.uniform(0,10, size=(100,))
y = (.4 * x +1 + delta)

In [5]:
# sudo code
input_size = 100
neural_network.compile(input_size)
neural_network.train(x, y, batch_size=20)

In [6]:
neural_network.validate(x, y)
neural_network.predict(x)

In [7]:
neural_network.architecture

(1, 10)